In [ ]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sklearn as sklearn
import seaborn as sns; sns.set()
import scipy
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib
import json
from tqdm import tqdm_notebook as tqdm
import os
from scipy import stats
import math
import scanpy as sc
import astropy
from astropy.convolution import convolve, Gaussian2DKernel
import mahotas as mh
from ipywidgets import interact, fixed

%pylab inline

In [ ]:
def pseudotime_per_bead(slide_seq_dge, barcode_locations, negative_gene_to_coefficient_dict, 
                                positive_gene_to_coefficient_dict, point_size, UMI_cutoff):
    barcode_locations_for_pseudo = barcode_locations.copy()
    barcode_locations_for_pseudo['Pseudotime'] = 0
    for i in negative_gene_to_coefficient_dict:
        try:
            barcode_locations_for_pseudo['Pseudotime'] = (barcode_locations_for_pseudo['Pseudotime'] + 
                                        slide_seq_dge.transpose()[i]*negative_gene_to_coefficient_dict.get(i))
        except:
            print('WARNING: The following gene: '+str(i)+
                  ' is missing in the slide seq dge and was not used in calculating pseudotime')
    for j in positive_gene_to_coefficient_dict:
        try:
            barcode_locations_for_pseudo['Pseudotime'] = (barcode_locations_for_pseudo['Pseudotime'] + 
                                        slide_seq_dge.transpose()[j]*positive_gene_to_coefficient_dict.get(j))
        except:
            print('WARNING: The following gene: '+str(j)+
                  ' is missing in the slide seq dge and was not used in calculating pseudotime')
            
    barcode_locations_for_pseudo['UMI'] = slide_seq_dge.sum(axis = 0)
    barcode_locations_for_pseudo = barcode_locations_for_pseudo[barcode_locations_for_pseudo.UMI > UMI_cutoff]
    barcode_locations_for_pseudo['Normalized Pseudotime'] = (barcode_locations_for_pseudo['Pseudotime']/
                                                            barcode_locations_for_pseudo['UMI'])
    matplotlib.rc('image', cmap='plasma')
    pseudotime_std_dev = float(barcode_locations_for_pseudo['Normalized Pseudotime'].describe()['std'])
    pseudotime_mean = float(barcode_locations_for_pseudo['Normalized Pseudotime'].describe()['mean'])
    vmin = pseudotime_mean - 2*pseudotime_std_dev
    vmax = pseudotime_mean + 2*pseudotime_std_dev
    fig = plt.figure(figsize=(15,10))
    plt.scatter(barcode_locations_for_pseudo['x'], barcode_locations_for_pseudo['y'], 
                c = barcode_locations_for_pseudo['Normalized Pseudotime'], s = point_size, 
                vmin = vmin, vmax = vmax)
    plt.colorbar()
    plt.savefig('Slide-seq_Pseudotime_Puck24.eps')
    plt.show()
    return barcode_locations_for_pseudo, fig

### Assign Pseudotime Value
    slide_seq_dge_file: slide seq differential gene expression matrix
    barcode_locations_file: slide seq barcodes with locations

In [ ]:
negative_gene_to_coefficient_dict = {'mt-Nd1': 2.240772116330779, 'Tuba3b': 20.0, 'Stmn1': 4.661355144844998, 'Cypt4': 2.914946084728282, 'mt-Cytb': 7.940721195057547, 'Hsp90aa1': 7.719926741335844}
positive_gene_to_coefficient_dict = {'Tnp2': 2.2300113564016115, 'Smcp': 20.0, 'Gsg1': 10.749147113890643, 'Oaz3': 13.684470608169942, 'Hmgb4': 11.19717467780924, 'Lyar': 3.205774497366639, 'Prm1': 1.8021648809899742, 'Dbil5': 2.4081140269931445}
slide_seq_dge_file = 'file name.csv' 
barcode_locations_file = 'file name.csv'

In [ ]:
slide_seq_dge = pd.read_csv(slide_seq_dge_file, header = 0, index_col = 0)
barcode_locations = pd.read_csv(barcode_locations_file, header = 0,index_col = 0)

In [ ]:
barcode_locations_for_pseudo = barcode_locations.copy()
barcode_locations_for_pseudo.head(3)

In [ ]:
# Visulize UMI distribution 
b = np.log2(barcode_locations_for_pseudo['UMI'])
plt.hist(x=b, color='#0504aa', bins= 50, alpha=0.7)
plt.grid(axis='y', alpha=0.75)
plt.xlabel('log2(UMI Count)')
plt.ylabel('Number of Beads')
plt.show()

In [ ]:
# Calculate the mean UMI count per bead
barcode_locations_for_pseudo['UMI'].mean()

In [ ]:
# Calculate pseudotime value for each bead
barcode_locations_for_pseudo['Pseudotime'] = 0
for i in negative_gene_to_coefficient_dict:
        try:
            barcode_locations_for_pseudo['Pseudotime'] = (barcode_locations_for_pseudo['Pseudotime'] + 
                                        slide_seq_dge[i]*negative_gene_to_coefficient_dict.get(i))
        except:
            print('WARNING: The following gene: '+str(i)+
                  ' is missing in the slide seq dge and was not used in calculating pseudotime')
for j in positive_gene_to_coefficient_dict:
        try:
            barcode_locations_for_pseudo['Pseudotime'] = (barcode_locations_for_pseudo['Pseudotime'] + 
                                        slide_seq_dge[j]*positive_gene_to_coefficient_dict.get(j))
        except:
            print('WARNING: The following gene: '+str(j)+
                  ' is missing in the slide seq dge and was not used in calculating pseudotime')   

In [ ]:
barcode_locations_for_pseudo['Normalized Pseudotime'] = (barcode_locations_for_pseudo['Pseudotime']/
                                                            barcode_locations_for_pseudo['UMI'])

In [ ]:
# Plot pseudotime values
barcodes_and_pseudotime, beads_pseudotime_plot = pseudotime_per_bead(slide_seq_dge = slide_seq_dge.T, barcode_locations=barcode_locations, 
                    negative_gene_to_coefficient_dict = negative_gene_to_coefficient_dict, 
                    positive_gene_to_coefficient_dict = positive_gene_to_coefficient_dict,
                    point_size = 6, UMI_cutoff = 30)

### Segmentation of the slide-seq data

In [ ]:
int_location = barcode_locations_for_pseudo.copy() 

In [ ]:
int_location = int_location.reset_index()
int_location.head()

In [ ]:
x=int_location['x'].values/25
y=int_location['y'].values/25

In [ ]:
#take only the integer part
int_location['xcoord_int'] = x.astype(int)
int_location['ycoord_int'] = y.astype(int)

In [ ]:
#Creat a 2-d array with values = 0
Matrix = np.zeros((256, 256))

In [ ]:
#Replace the array values with pseudotime value for each bead
for index, row in int_location.iterrows():
    Matrix[row['xcoord_int']-1][row['ycoord_int']-1] = row['Normalized Pseudotime']*1000

In [ ]:
pseudotime_std_dev = float(int_location['Normalized Pseudotime'].describe()['std'])
pseudotime_mean = float(int_location['Normalized Pseudotime'].describe()['mean'])
vmin = pseudotime_mean - 2*pseudotime_std_dev
vmax = pseudotime_mean + 2*pseudotime_std_dev

In [ ]:
# Plot psuedotime image
imgplot = plt.imshow(Matrix, cmap = 'Greys', vmin = vmin*1000, vmax = vmax*1000)
plt.colorbar()
plt.show()

In [ ]:
gauss_kernel = Gaussian2DKernel(2)
smoothed_Matrix = convolve(Matrix, gauss_kernel)
imgplot = plt.imshow(smoothed_Matrix, cmap = 'Greys') #vmin = vmin, vmax = vmax
plt.colorbar()
plt.show()

In [ ]:
#thresholding by an arbitrary number
binary_Matrix = smoothed_Matrix > 280
plt.imshow(binary_Matrix,cmap = 'Greys')

In [ ]:
labeled, nr_objects = mh.label(binary_Matrix)
print(nr_objects)

plt.imshow(labeled)
plt.jet()

In [ ]:
sigma = 2 #Sigma values can be changed
dnaf = mh.gaussian_filter(Matrix, sigma)
maxima = mh.regmax(mh.stretch(dnaf))
maxima,_= mh.label(maxima)
plt.imshow(maxima)

In [ ]:
# Distance transform
dist = mh.distance(binary_Matrix)
plt.imshow(dist)
plt.colorbar()

In [ ]:
dist = 500 - mh.stretch(dist)
watershed = mh.cwatershed(dist, maxima)
plt.imshow(watershed)

In [ ]:
# Watershed
watershed *= binary_Matrix
plt.imshow(watershed)

In [ ]:
Cluster = {}
for index, row in int_location.iterrows():
    Cluster[index] = watershed[row['xcoord_int']-1][row['ycoord_int']-1]

In [ ]:
Cluster_df = pd.DataFrame.from_dict(Cluster, orient='index')

In [ ]:
int_location['Cluster'] = Cluster_df[0]

### K-nearest neighbor

In [ ]:
Assigned = int_location[int_location['Cluster'] != 0]
Unassigned = int_location[int_location['Cluster'] == 0]

In [ ]:
X = Assigned.iloc[:, 6:8].values
y = Assigned.iloc[:, 8].values

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20)

In [ ]:
classifier = KNeighborsClassifier(n_neighbors=5)
classifier.fit(X_train, y_train)

In [ ]:
y_pred = classifier.predict(X_test)

In [ ]:
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

In [ ]:
X_real = Unassigned.iloc[:, 6:8].values
y_real = classifier.predict(X_real)

In [ ]:
y_real_df = pd.DataFrame(y_real) 

In [ ]:
Unassigned['Cluster'] = y_real_df[0].values

In [ ]:
int_location_seg = pd.concat([Unassigned, Assigned])

In [ ]:
Segmentation = int_location_seg.sort_index()

In [ ]:
Segmentation = Segmentation.set_index('barcode')

In [ ]:
grouped_seg = Segmentation.groupby('Cluster')
group_serise = {}

for name, group in grouped_seg:
    group_serise[name] = name   

In [ ]:
# Visulize Segmentation 
bool_col = Segmentation['Cluster']==0
plt.figure(figsize(6, 6))
for key, value in group_serise.items():
    boolcol = Segmentation['Cluster']==int(key)
    plt.scatter(Segmentation[boolcol]['x'], Segmentation[boolcol]['y'], s=15, alpha=0.7, cmap= 'tab20c') 
                    
plt.title('Segmented tubules')
plt.axis('equal')
plt.show()

### Output spatial gene matrix for every segmented seminiferous tubule

In [ ]:
Combined_raw = pd.concat([Segmentation, slide_seq_dge], axis =1)
Combined_raw.head()

In [ ]:
Combined_raw_drop = Combined_raw.drop(Combined_raw.loc[:, 'UMI':'ycoord_int'].columns, axis=1)

In [ ]:
grouped_raw = Combined_raw_drop.groupby('Cluster')

In [ ]:
Cluster_num = Combined_raw_drop['Cluster'].unique()
for i in tqdm(Cluster_num):
    new = grouped_raw.get_group(i)
    new.to_csv('Tubule_'+ str(i) +'.csv')